In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import itertools

# === Archivos TMY por país ===
archivos_tmy = {
    "Chile": ("tmy_-38.320_-71.960_2005_2023_pvwatts_final.csv", -4),
    "China": ("tmy_29.652_91.172_2005_2023_pvwatts_final.csv", +8),
    "Rusia": ("tmy_59.567_150.800_2005_2023_pvwatts_final.csv", +12)
}

# === Parámetros técnicos a evaluar ===
areas = [50000, 100000, 150000]                  # m² de campo solar
ef_opticas = [0.70, 0.75, 0.80]                  # eficiencia óptica
ef_termicas = [0.30, 0.35, 0.40]                 # eficiencia térmica
perdidas = [0.80, 0.85, 0.90]                    # factor de pérdidas
almacenamientos = [6, 10, 14]                   # horas de almacenamiento térmico

# === Potencia nominal constante para todas las simulaciones ===
potencia_nominal_mw = 35
horas_anuales = 8760

# === Crear combinaciones ===
combinaciones = list(itertools.product(areas, ef_opticas, ef_termicas, perdidas, almacenamientos))

# === Función para calcular producción CSP ===
def calcular_produccion(dni, area, eta_opt, eta_term, factor_perdidas):
    dni_kw_m2 = dni / 1000
    energia_capturada = dni_kw_m2 * area * eta_opt * factor_perdidas
    energia_electrica = energia_capturada * eta_term / 1000  # en MWh
    return energia_electrica

# === Resultados acumulados ===
resultados = []

# === Iterar por país y combinaciones ===
for pais, (archivo, utc_offset) in archivos_tmy.items():
    print(f"\nProcesando país: {pais}")
    
    # 1. Cargar datos
    df = pd.read_csv(archivo, skiprows=2)
    df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']].assign(Minute=0), errors='coerce')
    df['datetime_local'] = df['datetime'] + pd.to_timedelta(utc_offset, unit='h')
    df.set_index('datetime_local', inplace=True)

    # 2. Generar TMY promediando todas las horas de todos los años
    df['Month'] = df.index.month
    df['Day'] = df.index.day
    df['Hour'] = df.index.hour
    tmy_df = df.groupby(['Month', 'Day', 'Hour']).mean(numeric_only=True).reset_index()
    tmy_df['Year'] = df['Year'].max()
    tmy_df['datetime'] = pd.to_datetime(tmy_df[['Year', 'Month', 'Day', 'Hour']].assign(Minute=0))
    tmy_df.set_index('datetime', inplace=True)

    for area, eta_opt, eta_term, p, h_alm in combinaciones:
        produccion = tmy_df['DNI'].apply(lambda x: calcular_produccion(x, area, eta_opt, eta_term, p))
        produccion_total = produccion.sum()

        # Ajuste del factor de capacidad considerando almacenamiento térmico
        horas_extra = h_alm * 0.95  # suponemos 95% de uso eficiente de las horas de almacenamiento
        horas_equivalentes = horas_anuales + horas_extra * 365
        fc = (produccion_total / (potencia_nominal_mw * horas_equivalentes)) * 100

        resultados.append({
            "País": pais,
            "Área Colectora (m²)": area,
            "Eficiencia Óptica": eta_opt,
            "Eficiencia Térmica": eta_term,
            "Factor de Pérdidas": p,
            "Horas Almacenamiento": h_alm,
            "Producción Anual (MWh)": produccion_total,
            "Factor de Capacidad (%)": fc
        })

# === Exportar resultados ===
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("resultados_simulacion_csp_con_almacenamiento.csv", index=False)
print("\n✅ Simulaciones CSP finalizadas y guardadas en 'resultados_simulacion_csp_con_almacenamiento.csv'")
print(df_resultados.head())



Procesando país: Chile

Procesando país: China

Procesando país: Rusia

✅ Simulaciones CSP finalizadas y guardadas en 'resultados_simulacion_csp_con_almacenamiento.csv'
    País  Área Colectora (m²)  Eficiencia Óptica  Eficiencia Térmica  \
0  Chile                50000                0.7                 0.3   
1  Chile                50000                0.7                 0.3   
2  Chile                50000                0.7                 0.3   
3  Chile                50000                0.7                 0.3   
4  Chile                50000                0.7                 0.3   

   Factor de Pérdidas  Horas Almacenamiento  Producción Anual (MWh)  \
0                0.80                     6            17437.023912   
1                0.80                    10            17437.023912   
2                0.80                    14            17437.023912   
3                0.85                     6            18526.837906   
4                0.85                    1